## Sanjay Santokee - sanjay.santokee@my.uwi.edu


In [1]:
%pylab inline
%load_ext autoreload
%autoreload 2

Populating the interactive namespace from numpy and matplotlib


#### Importing Necessary Libraries

In [2]:
import copy
from DS import Queue, PriorityQueue, Stack
import numpy as np
import networkx as nx

#### Setting up State Class

In [3]:
class State(object):
    def __init__(self, city):
        self.city = city
        self.f = 0 # approx cost from start to goal through this state
        self.g = 0 # actual cost from start
        self.h = 0 # approx cost from goal
    
    
    # give string representation for easier display
    def __str__(self):
        return self.city
    
    # used to provide hashcode for hashing-based data structures like dictionaries and sets
    def __hash__(self):
        return hash(self.city)
    
    # need to give dummy implementation for priority queue based algorithms
    def __lt__(self, other):
        return 0

#### Setting up Graph

In [4]:
class Graph(object):
    def __init__(self, solution_state):
        self.solution_state = State(solution_state)
        
    # admissable heuristic on states
    def h(self, state):
        return distance_between(state.city, self.solution_state.city)
    
    # generate states that can be reached from another state 
    def get_edges(self, state):
        for _, adj_city, cost in map_graph.edges(nbunch=state.city, data='cost'):
            adj_state = State(adj_city)
            yield adj_state, cost

#### Admissable Heuristic (Straight-line distance)

In [5]:
def distance_between(city1, city2):
    x1, y1 = coords[city1]
    x2, y2 = coords[city2]
    return np.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

#### Function to Print Solution Route

In [6]:
def print_solution(parents, goal):
    
    route = []
    
    curr = goal
    stack = Stack()
    stack.push(curr)
    while parents[curr]:
        curr = parents[curr]
        stack.push(curr)
    while stack:
        route.append(stack.pop().city)
    print(' → '.join(route))

#### Function to Check if Romeo and Juliet are in the same city

In [7]:
def goal_test(r_current, j_current):
    return r_current.city == j_current.city

#### A* Function

In [8]:
def a_star(r_start_state, j_start_state, goal_test):
    
    # Setting up Romeo's Priority Queue, starting position, parents and explored set
    r_pqueue = PriorityQueue()
    r_pqueue.push(0, r_start_state)
    r_parents = {r_start_state: None}
    r_explored = set()
    
    # Setting up Juliet's Priority Queue, starting position, parents and explored set
    j_pqueue = PriorityQueue()
    j_pqueue.push(0, j_start_state)
    j_parents = {j_start_state: None}
    j_explored = set()
    
    
    # Setting Juliet's current state to be 'None' 
    # since a goal test must be done before updating 
    # Juliet's location so as to prevent the algorithm 
    # from skipping it and not finding the most optimal path
    
    j_current = State('None')
    
    while r_pqueue and j_pqueue:
        
        # Getting Romeo's current location
        i, r_current = r_pqueue.pop()
        r_explored.add(r_current)
        
        # Checking to see if both Romeo and Juliet are at the same city together        
        if goal_test(r_current, j_current):
            print('\n\nRomeo\'s route')
            print_solution(r_parents, r_current)
            print('\n\nJuliet\'s route')
            print_solution(j_parents, j_current)
            return True
        
        # Getting Juliet's current location
        i, j_current = j_pqueue.pop()
        j_explored.add(j_current)
        
        # Checking to see if both Romeo and Juliet are at the same city together
        if goal_test(r_current, j_current):
            print('\n\nRomeo\'s route')
            print_solution(r_parents, r_current)
            print('\n\nJuliet\'s route')
            print_solution(j_parents, j_current)
            return True
        
        # Updating Graphs with new goal cities
        # Romeo's goal would now be to reach Juliet's Location
        # Juliet's goal would now be to reach Romeo's Location
        r_graph = Graph(j_current.city)
        j_graph = Graph(r_current.city)
        
        # Utilizing Admissiable Heuristic (Straight-line distance)
        for r_adj_state, r_cost in r_graph.get_edges(r_current):
            g1 = r_current.g + r_cost
            h1 = r_graph.h(r_adj_state)
            f1 = g1 + h1
            if (r_adj_state in r_pqueue and r_pqueue[r_adj_state] > f1) or r_adj_state not in r_pqueue:
                r_adj_state.f = f1
                r_adj_state.g = g1
                r_adj_state.h = h1
                r_pqueue[r_adj_state] = f1
                r_parents[r_adj_state] = r_current
        
        # Utilizing Admissiable Heuristic (Straight-line distance)
        for j_adj_state, j_cost in j_graph.get_edges(j_current):
            g2 = j_current.g + j_cost
            h2 = j_graph.h(j_adj_state)
            f2 = g2 + h2
            if (j_adj_state in j_pqueue and j_pqueue[j_adj_state] > f2) or j_adj_state not in j_pqueue:
                j_adj_state.f = f2
                j_adj_state.g = g2
                j_adj_state.h = h2
                j_pqueue[j_adj_state] = f2
                j_parents[j_adj_state] = j_current
                
    print('No Solution found')
    return False

#### Reading in the Graph of cities and their traversal costs

In [9]:
map_graph = nx.Graph()
with open('cities.csv', 'r') as fp:
    lines = list(fp)
    lines = map(lambda x: x.strip().split(','), lines)
    for frm, to, cost in lines:
        map_graph.add_edge(u_of_edge=frm, v_of_edge=to, cost=float(cost))

#### Reading in the Coordinates of the Cities

In [10]:
coords = {}

with open('coords.csv', 'r') as fp:
    lines = list(fp)
    lines = map(lambda x: x.strip().split(','), lines)
    for loc, x, y in lines:
        coords[loc] = (float(x), float(y))
        

#### Viewing all of the Cities in the Graph

In [11]:
map_graph.nodes()

NodeView(('Ipswich', 'Swindon', 'Rochdale', 'Bexley', 'Brighton', 'Sutton', 'Stevenage', 'Gillingham', 'Stoke-on-Trent', 'Watford', 'Poole', 'Stockport', 'Eastbourne', 'Doncaster', 'Grays', 'Cardiff', 'Huddersfield', 'Maidstone', 'Oxford', 'Mansfield', 'Colchester', 'Worthing', 'Edinburgh', 'Wigan', 'Harlow', 'Luton', 'London', 'Dagenham', 'Dudley', 'Walsall', 'Glasgow', 'Darlington', 'Grimsby', 'Islington', 'Southport', 'Hastings', 'Woking', 'Manchester', 'Belfast', 'Plymouth', 'Bolton', 'Solihull', 'Telford', 'Reading', 'Northampton', 'Chesterfield', 'Warrington', 'Bath', 'Croydon', 'Peterborough', 'Basildon', 'Blackpool', 'Newport', 'Blackburn', 'Cambridge', 'Sunderland', 'Southampton', 'York', 'Swansea', 'Norwich', 'Portsmouth', 'Bournemouth', 'Worcester', 'Coventry', 'Nuneaton', 'Romford', 'Crawley', 'Archway', 'Leicester', 'Nottingham', 'Liverpool', 'Wembley', 'Burnley', 'Preston', 'Becontree', 'Harrogate', 'Aberdeen', 'Sheffield', 'Derby', 'Bristol', 'Slough', 'Chelmsford', 'Lin

#### Viewing all of the Edges from London in the Graph

In [12]:
map_graph.edges(nbunch='London', data='cost')

EdgeDataView([('London', 'Wigan', 911.3692756831273), ('London', 'Liverpool', 1018.1057181986507), ('London', 'Lincoln', 539.2857371928465), ('London', 'Leeds', 650.5713032241687), ('London', 'Cambridge', 177.89646833707764), ('London', 'Edinburgh', 1632.0004152200859), ('London', 'Southampton', 422.27730939002726), ('London', 'Chester', 972.8471351399976), ('London', 'Dundee', 763.4470906072128), ('London', 'Rotherham', 342.73398696283937), ('London', 'Belfast', 897.1224710447809), ('London', 'Bexley', 84.75216020119203)])

#### Setting the Starting Locations for Romeo and Juliet

In [13]:
r_start_state = State('London')
j_start_state = State('Edinburgh')

#### Running A*

In [14]:
a_star(r_start_state, j_start_state, goal_test)



Romeo's route
London → Bexley → Reading → Stevenage


Juliet's route
Edinburgh → Stevenage


True

***
#### References

Mykel J. Kochenderfer, Tim A. Wheeler. 2019. Algorithms for Optimization. Cambridge, Massachusetts: The MIT Press.